In [1]:
library(edgeR)
library(magrittr)
library(ggplot2)
library(DESeq2)

Loading required package: limma

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:limma’:

    plotMA


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tap

# Automatise

# On cell-state level

## For LV

In [2]:
x=read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALL_CELLSTATE_PSEUDOBULK_LV.csv")
genes_tofilter=read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALL_CELLSTATE_PSEUDOBULK_FILTERING_LV.csv")
colnames(genes_tofilter) <- gsub("mutation.negative", 'PVneg', colnames(genes_tofilter))

# Only needed for the column cell_state
CELLTYPE_STATE <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/CELLSTATE_TRANSLATION_TABLE.csv")
colnames(CELLTYPE_STATE) <- c("cell_states", 'cell_type')

# Removes patients with only few (<10) nuclei numbers (as they might bias the analysis)
CELLTYPE_FILTER <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ABSOLUTE_CELLSTATES_NUMBER_LV.csv")


rownames(x) <- x$X
x <- x[,-1]
l <- strsplit(colnames(x), "__")

In [3]:
# This steps are needed to make names equal - space and + was converted to .
#CELLTYPE_FILTER$cell_states <- gsub("\\+", "\\.", CELLTYPE_FILTER$cell_states)
#CELLTYPE_FILTER$cell_states <- gsub(" ", "\\.", CELLTYPE_FILTER$cell_states)
#CELLTYPE_FILTER$cell_states <- gsub("\\-", "\\.", CELLTYPE_FILTER$cell_states)

In [4]:
meta.data <- t(as.data.frame(l))
rownames(meta.data) <- colnames(x)

meta.data <- as.data.frame(meta.data)
colnames(meta.data)  <-  c("Primary_Genetic_Phenotype", "cell_state", 'Patient', 'X10X_version', 'Sex')

In [8]:
unique(meta.data$cell_state[which(!(meta.data$cell_state %in% CELLTYPE_FILTER[,1]))])

factor(0)
71 Levels: AD1.0 AD1.1 AD2 AD3 B CD4T_act CD4T_naive CD4T_reg ... vFB4

Check that read counts are natural numbers

In [12]:
# Numbers of observations in total for Adipocytes, LMNA in RV
CELLTYPE_FILTER[1,as.character(unique(meta.data[which(meta.data[,"Primary_Genetic_Phenotype"]=="TTN"),"Patient"]))]

,DT4,H14,H16,H36,H40,IC_H02,H37,H41,H34,H35,H39,IC_H01
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,7,1,2,1,1,1,0,0,0,0,0,0


In [13]:
# Numbers of observations in total for Adipocytes, control in RV
CELLTYPE_FILTER[1,as.character(unique(meta.data[which(meta.data[,"Primary_Genetic_Phenotype"]=="control"),"Patient"]))]

,D1,D2,D4,D5,D6,D7,ED_H15,ED_H20,ED_H25,ED_H26,ED_H35,H49,H51,H53,ED_H37,H46,H55,H67
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,54,71,109,66,25,3,21,29,6,52,22,12,7,1,0,0,0,0


In [14]:
head(meta.data)

,Primary_Genetic_Phenotype,cell_state,Patient,X10X_version,Sex
,<fct>,<fct>,<fct>,<fct>,<fct>
LMNA__AD1.0__DL2__V3__m,LMNA,AD1.0,DL2,V3,m
LMNA__AD1.0__H05__V3__m,LMNA,AD1.0,H05,V3,m
LMNA__AD1.0__H26__V3__f,LMNA,AD1.0,H26,V3,f
LMNA__AD1.0__H28__V3__m,LMNA,AD1.0,H28,V3,m
LMNA__AD1.0__H31__V3__m,LMNA,AD1.0,H31,V3,m
LMNA__AD1.1__H05__V3__m,LMNA,AD1.1,H05,V3,m


In [15]:
GENOTYPES <- unique(meta.data[,"Primary_Genetic_Phenotype"][meta.data[,"Primary_Genetic_Phenotype"] != "control"])
GENOTYPES

[1] LMNA  PKP2  PVneg RBM20 TTN  
Levels: control LMNA PKP2 PVneg RBM20 TTN

In [16]:
CELL_LEVEL <- "CELLSTATES"
REGION <- "LV"

In [17]:
##########
# Convert to bool
CELLTYPE_FILTER <- CELLTYPE_FILTER
CELLTYPE_FILTER[,2:ncol(CELLTYPE_FILTER)] <- apply(CELLTYPE_FILTER[,-1], 2, function(i) i>=5)

In [18]:
unique(meta.data[which(meta.data[,"Primary_Genetic_Phenotype"]=="TTN"),"Patient"])

[1] DT4    H14    H16    H36    H40    IC_H02 H37    H41    H34    H35   
[11] H39    IC_H01
64 Levels: D1 D2 D4 D5 D6 D7 DL2 DL3 DT4 ED_H15 ED_H20 ED_H25 ED_H26 ... IC_H04

In [19]:
# This is the example from above, they will be excluded in downstream analysis
# In this case, the comparison will be excluded, as only one sample has enough observations
CELLTYPE_FILTER[1,as.character(unique(meta.data[which(meta.data[,"Primary_Genetic_Phenotype"]=="TTN"),"Patient"]))]

,DT4,H14,H16,H36,H40,IC_H02,H37,H41,H34,H35,H39,IC_H01
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [ ]:
i <- 1

for(GENOTYPE in as.character(GENOTYPES)){
    message("\n###START: ", GENOTYPE," #####\n")
    
    for(CELL_STATE in as.character(unique(meta.data$cell_state))){
        
        CONTROL_COLUMN <- paste("control_",CELL_STATE, sep="")
        GENOTYPE_COLUMN <- paste(GENOTYPE,"_", CELL_STATE, sep="")
        CELLTYPE <- as.character(CELLTYPE_STATE[which(CELLTYPE_STATE$cell_states==CELL_STATE),"cell_type"])
        
        
        x_sub <- x[,which(meta.data[,"Primary_Genetic_Phenotype"] %in% c("control", GENOTYPE))]
        meta.data_sub <- meta.data[which(meta.data[,"Primary_Genetic_Phenotype"] %in% c("control", GENOTYPE)),]
        
        x_sub <- x_sub[,which(meta.data_sub$cell_state %in% c(CELL_STATE))]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$cell_state %in% c(CELL_STATE)),]
        
        ##### remove first line (cell_type column)
        CELLTYPE_FILTER_SUB <- CELLTYPE_FILTER
        PATIENTS_TOKEEP <- CELLTYPE_FILTER_SUB[which(CELLTYPE_FILTER_SUB$cell_states==CELL_STATE),-1]
        PATIENTS_TOKEEP <- colnames(PATIENTS_TOKEEP)[which(as.logical(PATIENTS_TOKEEP))]
        #####
        
        # Subset only for patients which fulfill condition
        x_sub <- x_sub[,which(meta.data_sub$Patient %in% PATIENTS_TOKEEP)]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$Patient %in% PATIENTS_TOKEEP),]
        meta.data_sub[,"Primary_Genetic_Phenotype"] = droplevels(meta.data_sub[,"Primary_Genetic_Phenotype"]) ## Drop factor levels
   
        
        # The test is only run, if 2 conditions have at least 2 patients
        if (length(unique(meta.data_sub[,"Primary_Genetic_Phenotype"]))==2 & 
            all(table(as.character(meta.data_sub[,"Primary_Genetic_Phenotype"]))>=3))
           {
             # Prepare DGEList object
            meta.data_sub[,"Primary_Genetic_Phenotype"] <- as.factor(as.character(meta.data_sub[,"Primary_Genetic_Phenotype"]))
            dge <- DGEList(counts=x_sub, group=meta.data_sub[,"Primary_Genetic_Phenotype"])
            
            # Filter genes based on expression, the qlf Object will be filtered to exclude them from FDR calculation
            genes_tofilter_sub_GENOTYPE <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% GENOTYPE &
                                                                 gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE & 
                                                                 gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP)]

            genes_tofilter_sub_CONTROL <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% 'control' &
                                                                gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE & 
                                                                gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP)]
            
            genes_tofilter_sub <- data.frame("X"=genes_tofilter[,"X"],
                                             CONTROL_COLUMN=apply(genes_tofilter_sub_CONTROL, 1, mean),
                                             GENOTYPE_COLUMN=apply(genes_tofilter_sub_GENOTYPE, 1, mean))
            colnames(genes_tofilter_sub) <- c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)
            
            keep <- which(genes_tofilter_sub[,CONTROL_COLUMN] > 0.0125 |
                          genes_tofilter_sub[,GENOTYPE_COLUMN] > 0.0125)
            
            #keep <- genes_tofilter[,CONTROL_COLUMN]>0.0125 | ##### change & to | ##### 
            #        genes_tofilter[,GENOTYPE_COLUMN]>0.0125
            #dge <- dge[keep, , keep.lib.sizes=FALSE]
    
            # PP, model matrix, https://www.nature.com/articles/nmeth.4612 (edgeRQLFDetRate)
            dge <- calcNormFactors(dge)
            cdr <- scale(colMeans(x_sub > 0))
            design <- model.matrix(~ cdr + meta.data_sub[,"Primary_Genetic_Phenotype"])
    
            # 
            dge <- estimateDisp(dge, design = design)
            fit <- glmQLFit(dge, design = design)
            qlf <- glmQLFTest(fit)
    
            # For all
            tt <- topTags(qlf, n = Inf)
            
            # Only for "expressed/detected" genes
            tt_filtered <- topTags(qlf[keep,], n = Inf)
            
            tt$table[,"Gene"] <- as.character(rownames(tt$table))
            tt_filtered$table[,"Gene"] <- as.character(rownames(tt_filtered$table))
            tt_filtered$table[,"low_expression"] <- "F" #####  Low-expression column ##### 
            tt_filtered$table[,"FDR_plot"] <- -log10(tt_filtered$table$FDR) #####  For plotting Volcano, here we use FDR, not pValue  ##### 
            
            
            tt_merged <- merge(tt$table[,c("Gene", "logFC", "logCPM", 'F', 'PValue')], 
                               tt_filtered$table[,c("Gene", "FDR", "low_expression", "FDR_plot")], 
                               by="Gene", all=T)   
            
            # Replace NA FDRs with NA                   #####  ORDER CHANGED #####  
            tt_merged[which(is.na(tt_merged$FDR)),"FDR"] <- 1 
            
            EXPRESSION_MEAN <- genes_tofilter_sub[,c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)]
            colnames(EXPRESSION_MEAN) <- c("Gene", "mean_exp_control", 'mean_exp_genotype')
            tt_merged <- merge(tt_merged, EXPRESSION_MEAN, by="Gene")
    
            
            tt_merged[,"Region"] <- REGION
            tt_merged[,"annotation_level"] <- CELL_LEVEL
            tt_merged[,"cell_state"] <- CELL_STATE
            tt_merged[,"cell_type"] <- CELLTYPE
            tt_merged[,"comparison"] <- paste("control_", GENOTYPE, sep="")
            
            ###### Suggestion to add those columns
            tt_merged[,"Observations_genotype"] <- as.numeric(table(meta.data_sub[,"Primary_Genetic_Phenotype"])[GENOTYPE])
            tt_merged[,"Observations_reference"] <- as.numeric(table(meta.data_sub[,"Primary_Genetic_Phenotype"])['control'])
            
            # low expression add T
            tt_merged[which(is.na(tt_merged$low_expression)),"low_expression"] <- "T"
            
            if(i==1){
                 final_df <- tt_merged   
            } else {
                final_df <- rbind(final_df, tt_merged)
            }
            
            i <- i + 1
            message("\n###FINISHED: ", CELL_STATE," #####\n")
        } else {
            message("\n###Skip: ", CELL_STATE," #####\n")
        }
    }
}


###START: LMNA #####



###Skip: AD1.0 #####



###FINISHED: AD1.1 #####



###FINISHED: AD2 #####



###Skip: B #####



###FINISHED: CD4T_act #####



###FINISHED: CD4T_naive #####



###Skip: CD4T_reg #####



###FINISHED: CD8T_cytox #####



###FINISHED: CD8T_em #####



###Skip: CD8T_te_IFNGhi #####



###FINISHED: CD8T_te #####



###FINISHED: CD8T_trans #####



###FINISHED: EC1.0 #####



###FINISHED: EC2.0 #####



###FINISHED: EC5.0 #####



###FINISHED: EC6.0 #####



###FINISHED: EC7.0 #####



###FINISHED: EC8.0 #####



###FINISHED: ILC #####



###FINISHED: MAIT.like #####



###FINISHED: MO_CD16. #####



###FINISHED: MO_VCAN. #####



###FINISHED: MP_FOLR2 #####



###FINISHED: MP_ISG #####



###FINISHED: MP_LYVE1hi.MHCII_intermediate #####



###FINISHED: MP_LYVE1hi.MHCII_lo #####



###FINISHED: MP_LYVE1lo.MHCII_hi #####



###FINISHED: MP_NFkB #####



###FINISHED: MP_OSM #####



###FINISHED: MP_TREM2. #####



###FINISHED: MP_transitional #####



###FINISHED: M

In [28]:
tail(final_df)

,Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison,Observations_genotype,Observations_reference
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
9256483,ZXDB,-0.8793595,2.646900,2.3759966,0.12967598,1.0000000,T,NA,0.004488071,0.002088314,LV,CELLSTATES,vFB4,FB,control_TTN,12,18
9256484,ZXDC,0.1884164,6.614096,1.5478392,0.21940012,0.4574654,F,0.3396418,0.181390447,0.269253461,LV,CELLSTATES,vFB4,FB,control_TTN,12,18
9256485,ZYG11A,-0.1996221,2.986955,0.1418047,0.70812656,1.0000000,T,NA,0.010091202,0.005229333,LV,CELLSTATES,vFB4,FB,control_TTN,12,18
9256486,ZYG11B,0.2527605,6.283061,2.7807413,0.10181252,0.2921935,F,0.5343295,0.141435339,0.214863723,LV,CELLSTATES,vFB4,FB,control_TTN,12,18
9256487,ZYX,-0.7261541,4.436926,5.3063500,0.02555078,0.1227531,F,0.9109676,0.041143998,0.037142365,LV,CELLSTATES,vFB4,FB,control_TTN,12,18
9256488,ZZEF1,0.1289970,6.690831,0.7087126,0.40398010,0.6458588,F,0.1898624,0.199194105,0.280689411,LV,CELLSTATES,vFB4,FB,control_TTN,12,18


In [29]:
table(final_df[,'cell_state'])


                        AD1.1                           AD2 
                        67076                         33538 
                     CD4T_act                    CD4T_naive 
                       167690                        167690 
                   CD8T_cytox                       CD8T_em 
                       167690                        167690 
                      CD8T_te                    CD8T_trans 
                       167690                        167690 
                         cDC2                         EC1.0 
                       167690                        167690 
                        EC2.0                         EC5.0 
                       167690                        167690 
                        EC6.0                         EC7.0 
                       167690                        167690 
                        EC8.0                           ILC 
                       167690                         33538 
                    MAI

In [30]:
tapply(final_df[,"FDR"], 
       final_df[,'cell_state'], 
       min)

AD1.1                           AD2 
                 5.133917e-14                  5.283773e-10 
                     CD4T_act                    CD4T_naive 
                 6.209328e-19                  1.237222e-14 
                   CD8T_cytox                       CD8T_em 
                 5.492983e-10                  3.090215e-08 
                      CD8T_te                    CD8T_trans 
                 1.447958e-22                  2.898908e-08 
                         cDC2                         EC1.0 
                 5.121410e-11                  5.253096e-10 
                        EC2.0                         EC5.0 
                 9.399432e-07                  1.402898e-09 
                        EC6.0                         EC7.0 
                 2.621271e-09                  2.963317e-19 
                        EC8.0                           ILC 
                 4.077692e-13                  4.764852e-01 
                    MAIT.like                          Mast 
                 4.085495e-07                  3.548006e-19 
                     MO_CD16.                      MO_VCAN. 
                 2.596590e-05                  1.544289e-20 
                     MP_FOLR2                        MP_ISG 
                 1.479858e-28                  3.171914e-27 
MP_LYVE1hi.MHCII_intermediate           MP_LYVE1hi.MHCII_lo 
                 4.892239e-23                  1.317630e-10 
          MP_LYVE1lo.MHCII_hi                       MP_NFkB 
                 4.411819e-26                  3.180161e-09 
                       MP_OSM               MP_transitional 
                 4.887047e-11                  8.202326e-24 
                    MP_TREM2.                         NC1.0 
                 3.856987e-16                  1.994132e-10 
                        NC1.1                         NC1.3 
                 1.719549e-21                  3.667633e-03 
                          NC2                           NC3 
                 4.493433e-14                  5.882641e-07 
                          NC5                     NK_CD16hi 
                 5.031252e-03                  1.427535e-31 
                    NK_CD56hi                           PC1 
                 6.456235e-13                  2.344014e-13 
                          PC2                           PC3 
                 3.319375e-31                  1.003460e-09 
              prolif_Myeloids                        SMC1.1 
                 1.647370e-20                  1.540019e-15 
                       SMC1.2                          SMC2 
                 8.121801e-21                  1.234224e-27 
                       vCM1.0                        vCM1.1 
                 3.058072e-07                  3.376155e-09 
                       vCM1.2                        vCM1.3 
                 1.670289e-10                  3.636985e-09 
                         vCM2                        vCM3.0 
                 2.386335e-07                  2.487216e-07 
                       vCM3.1                          vCM4 
                 4.595156e-09                  1.404860e-08 
                         vCM5                        vFB1.0 
                 6.104807e-20                  7.663469e-10 
                       vFB1.1                        vFB1.2 
                 5.608803e-09                  4.857327e-14 
                         vFB2                          vFB3 
                 2.642222e-11                  3.070947e-08 
                         vFB4 
                 7.316290e-12

In [31]:
write.csv(final_df, "/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALLCELLSTATES_ALLGENOTYPES_EDGER_LV_V8.csv")

In [46]:
#final_df <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALLCELLSTATES_ALLGENOTYPES_EDGER_LV_V3.csv")

## For RV

In [32]:
x=read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALL_CELLSTATE_PSEUDOBULK_RV.csv")
genes_tofilter=read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALL_CELLSTATE_PSEUDOBULK_FILTERING_RV.csv")
colnames(genes_tofilter) <- gsub("mutation.negative", 'PVneg', colnames(genes_tofilter))

# Only needed for the column cell_state
CELLTYPE_STATE <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/CELLSTATE_TRANSLATION_TABLE.csv")
colnames(CELLTYPE_STATE) <- c("cell_states", 'cell_type')

# Removes patients with only few (<10) nuclei numbers (as they might bias the analysis)
CELLTYPE_FILTER <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ABSOLUTE_CELLSTATES_NUMBER_RV.csv")


rownames(x) <- x$X
x <- x[,-1]
l <- strsplit(colnames(x), "__")

In [33]:
# This steps are needed to make names equal - space and + was converted to .
#CELLTYPE_FILTER$cell_states <- gsub("\\+", "\\.", CELLTYPE_FILTER$cell_states)
#CELLTYPE_FILTER$cell_states <- gsub(" ", "\\.", CELLTYPE_FILTER$cell_states)
#CELLTYPE_FILTER$cell_states <- gsub("\\-", "\\.", CELLTYPE_FILTER$cell_states)

In [35]:
meta.data <- t(as.data.frame(l))
rownames(meta.data) <- colnames(x)

meta.data <- as.data.frame(meta.data)
colnames(meta.data)  <-  c("Primary_Genetic_Phenotype", "cell_state", 'Patient', 'X10X_version', 'Sex')

In [40]:
# Numbers of observations in total for Adipocytes, LMNA in RV
CELLTYPE_FILTER[1,as.character(unique(meta.data[which(meta.data[,"Primary_Genetic_Phenotype"]=="TTN"),"Patient"]))]

,H36,H14,H39,H41,H34,H35
,<int>,<int>,<int>,<int>,<int>,<int>
1,1,0,0,0,0,0


In [41]:
# Numbers of observations in total for Adipocytes, control in RV
CELLTYPE_FILTER[1,as.character(unique(meta.data[which(meta.data[,"Primary_Genetic_Phenotype"]=="control"),"Patient"]))]

,D1,D2,D4,D5,D6,D7,ED_H20,ED_H26,ED_H35,H49,H53,H55,ED_H15,ED_H25,ED_H37
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,10,2,36,3,12,12,41,42,6,67,9,4,0,0,0


In [42]:
head(meta.data)

,Primary_Genetic_Phenotype,cell_state,Patient,X10X_version,Sex
,<fct>,<fct>,<fct>,<fct>,<fct>
LMNA__AD1.0__H06__V3__f,LMNA,AD1.0,H06,V3,f
LMNA__AD1.0__H24__V3__f,LMNA,AD1.0,H24,V3,f
LMNA__AD1.0__H28__V3__m,LMNA,AD1.0,H28,V3,m
LMNA__AD1.1__H06__V3__f,LMNA,AD1.1,H06,V3,f
LMNA__AD1.1__H24__V3__f,LMNA,AD1.1,H24,V3,f
LMNA__AD1.1__H25__V3__m,LMNA,AD1.1,H25,V3,m


In [43]:
GENOTYPES <- unique(meta.data[,"Primary_Genetic_Phenotype"][meta.data[,"Primary_Genetic_Phenotype"] != "control"])
GENOTYPES

[1] LMNA  PKP2  PVneg RBM20 TTN  
Levels: control LMNA PKP2 PVneg RBM20 TTN

In [44]:
CELL_LEVEL <- "CELLSTATES"
REGION <- "RV"

In [45]:
##########
# Convert to bool
CELLTYPE_FILTER <- CELLTYPE_FILTER
CELLTYPE_FILTER[,2:ncol(CELLTYPE_FILTER)] <- apply(CELLTYPE_FILTER[,-1], 2, function(i) i>=5)

In [46]:
unique(meta.data[which(meta.data[,"Primary_Genetic_Phenotype"]=="TTN"),"Patient"])

[1] H36 H14 H39 H41 H34 H35
42 Levels: D1 D2 D4 D5 D6 D7 ED_H15 ED_H20 ED_H25 ED_H26 ED_H35 ED_H37 ... IC_H04

In [47]:
# This is the example from above, they will be excluded in downstream analysis
# In this case, the comparison will be excluded, as only one sample has enough observations
CELLTYPE_FILTER[1,as.character(unique(meta.data[which(meta.data[,"Primary_Genetic_Phenotype"]=="TTN"),"Patient"]))]

,H36,H14,H39,H41,H34,H35
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE


In [48]:
i <- 1

for(GENOTYPE in as.character(GENOTYPES)){
    message("\n###START: ", GENOTYPE," #####\n")
    
    for(CELL_STATE in as.character(unique(meta.data$cell_state))){
        
        CONTROL_COLUMN <- paste("control_",CELL_STATE, sep="")
        GENOTYPE_COLUMN <- paste(GENOTYPE,"_", CELL_STATE, sep="")
        CELLTYPE <- as.character(CELLTYPE_STATE[which(CELLTYPE_STATE$cell_states==CELL_STATE),"cell_type"])
        
        
        x_sub <- x[,which(meta.data[,"Primary_Genetic_Phenotype"] %in% c("control", GENOTYPE))]
        meta.data_sub <- meta.data[which(meta.data[,"Primary_Genetic_Phenotype"] %in% c("control", GENOTYPE)),]
        
        x_sub <- x_sub[,which(meta.data_sub$cell_state %in% c(CELL_STATE))]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$cell_state %in% c(CELL_STATE)),]
        
        ##### remove first line (cell_type column)
        CELLTYPE_FILTER_SUB <- CELLTYPE_FILTER
        PATIENTS_TOKEEP <- CELLTYPE_FILTER_SUB[which(CELLTYPE_FILTER_SUB$cell_states==CELL_STATE),-1]
        PATIENTS_TOKEEP <- colnames(PATIENTS_TOKEEP)[which(as.logical(PATIENTS_TOKEEP))]
        #####
        
        # Subset only for patients which fulfill condition
        x_sub <- x_sub[,which(meta.data_sub$Patient %in% PATIENTS_TOKEEP)]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$Patient %in% PATIENTS_TOKEEP),]
        meta.data_sub[,"Primary_Genetic_Phenotype"] = droplevels(meta.data_sub[,"Primary_Genetic_Phenotype"]) ## Drop factor levels
   
        
        # The test is only run, if 2 conditions have at least 2 patients
        if (length(unique(meta.data_sub[,"Primary_Genetic_Phenotype"]))==2 & 
            all(table(as.character(meta.data_sub[,"Primary_Genetic_Phenotype"]))>=3))
           {
             # Prepare DGEList object
            meta.data_sub[,"Primary_Genetic_Phenotype"] <- as.factor(as.character(meta.data_sub[,"Primary_Genetic_Phenotype"]))
            dge <- DGEList(counts=x_sub, group=meta.data_sub[,"Primary_Genetic_Phenotype"])
            
            # Filter genes based on expression, the qlf Object will be filtered to exclude them from FDR calculation
            genes_tofilter_sub_GENOTYPE <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% GENOTYPE &
                                                                 gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE & 
                                                                 gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP)]

            genes_tofilter_sub_CONTROL <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% 'control' &
                                                                gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE & 
                                                                gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP)]
            
            genes_tofilter_sub <- data.frame("X"=genes_tofilter[,"X"],
                                             CONTROL_COLUMN=apply(genes_tofilter_sub_CONTROL, 1, mean),
                                             GENOTYPE_COLUMN=apply(genes_tofilter_sub_GENOTYPE, 1, mean))
            colnames(genes_tofilter_sub) <- c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)
            
            keep <- which(genes_tofilter_sub[,CONTROL_COLUMN] > 0.0125 |
                          genes_tofilter_sub[,GENOTYPE_COLUMN] > 0.0125)
            
            #keep <- genes_tofilter[,CONTROL_COLUMN]>0.0125 | ##### change & to | ##### 
            #        genes_tofilter[,GENOTYPE_COLUMN]>0.0125
            #dge <- dge[keep, , keep.lib.sizes=FALSE]
    
            # PP, model matrix, https://www.nature.com/articles/nmeth.4612 (edgeRQLFDetRate)
            dge <- calcNormFactors(dge)
            cdr <- scale(colMeans(x_sub > 0))
            design <- model.matrix(~ cdr + meta.data_sub[,"Primary_Genetic_Phenotype"])
    
            # 
            dge <- estimateDisp(dge, design = design)
            fit <- glmQLFit(dge, design = design)
            qlf <- glmQLFTest(fit)
    
            # For all
            tt <- topTags(qlf, n = Inf)
            
            # Only for "expressed/detected" genes
            tt_filtered <- topTags(qlf[keep,], n = Inf)
            
            tt$table[,"Gene"] <- as.character(rownames(tt$table))
            tt_filtered$table[,"Gene"] <- as.character(rownames(tt_filtered$table))
            tt_filtered$table[,"low_expression"] <- "F" #####  Low-expression column ##### 
            tt_filtered$table[,"FDR_plot"] <- -log10(tt_filtered$table$FDR) #####  For plotting Volcano, here we use FDR, not pValue  ##### 
            
            
            tt_merged <- merge(tt$table[,c("Gene", "logFC", "logCPM", 'F', 'PValue')], 
                               tt_filtered$table[,c("Gene", "FDR", "low_expression", "FDR_plot")], 
                               by="Gene", all=T)   
            
            # Replace NA FDRs with NA                   #####  ORDER CHANGED #####  
            tt_merged[which(is.na(tt_merged$FDR)),"FDR"] <- 1 
            
            EXPRESSION_MEAN <- genes_tofilter_sub[,c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)]
            colnames(EXPRESSION_MEAN) <- c("Gene", "mean_exp_control", 'mean_exp_genotype')
            tt_merged <- merge(tt_merged, EXPRESSION_MEAN, by="Gene")
    
            
            tt_merged[,"Region"] <- REGION
            tt_merged[,"annotation_level"] <- CELL_LEVEL
            tt_merged[,"cell_state"] <- CELL_STATE
            tt_merged[,"cell_type"] <- CELLTYPE
            tt_merged[,"comparison"] <- paste("control_", GENOTYPE, sep="")
            
            ###### Suggestion to add those columns
            tt_merged[,"Observations_genotype"] <- as.numeric(table(meta.data_sub[,"Primary_Genetic_Phenotype"])[GENOTYPE])
            tt_merged[,"Observations_reference"] <- as.numeric(table(meta.data_sub[,"Primary_Genetic_Phenotype"])['control'])
            
            # low expression add T
            tt_merged[which(is.na(tt_merged$low_expression)),"low_expression"] <- "T"
            
            if(i==1){
                 final_df <- tt_merged   
            } else {
                final_df <- rbind(final_df, tt_merged)
            }
            
            i <- i + 1
            message("\n###FINISHED: ", CELL_STATE," #####\n")
        } else {
            message("\n###Skip: ", CELL_STATE," #####\n")
        }
    }
}


###START: LMNA #####



###Skip: AD1.0 #####



###Skip: AD1.1 #####



###FINISHED: AD2 #####



###Skip: B #####



###FINISHED: CD4T_act #####



###FINISHED: CD4T_naive #####



###Skip: CD4T_reg #####



###Skip: CD8T_cytox #####



###FINISHED: CD8T_em #####



###Skip: CD8T_te_IFNGhi #####



###FINISHED: CD8T_te #####



###FINISHED: CD8T_trans #####



###FINISHED: EC1.0 #####



###FINISHED: EC2.0 #####



###FINISHED: EC5.0 #####



###FINISHED: EC6.0 #####



###FINISHED: EC7.0 #####



###FINISHED: EC8.0 #####



###Skip: ILC #####



###Skip: MAIT.like #####



###Skip: MO_CD16. #####



###FINISHED: MO_VCAN. #####



###FINISHED: MP_FOLR2 #####



###FINISHED: MP_ISG #####



###FINISHED: MP_LYVE1hi.MHCII_intermediate #####



###FINISHED: MP_LYVE1hi.MHCII_lo #####



###FINISHED: MP_LYVE1lo.MHCII_hi #####



###Skip: MP_NFkB #####



###FINISHED: MP_OSM #####



###FINISHED: MP_TREM2. #####



###FINISHED: MP_transitional #####



###Skip: Mast #####



###Skip: Meso #

In [49]:
tail(final_df)

,Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison,Observations_genotype,Observations_reference
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
7982039,ZXDB,0.2745719,3.865191,0.1260335,0.750507607,1.00000000,T,NA,0.0025263358,0.006797264,RV,CELLSTATES,vFB4,FB,control_TTN,6,14
7982040,ZXDC,0.2828352,6.474848,0.8021216,0.372774272,0.69410837,F,0.1585727,0.1454636983,0.231421238,RV,CELLSTATES,vFB4,FB,control_TTN,6,14
7982041,ZYG11A,0.9231232,3.793061,1.4493956,0.394627470,1.00000000,T,NA,0.0001649621,0.005780245,RV,CELLSTATES,vFB4,FB,control_TTN,6,14
7982042,ZYG11B,0.2472182,6.315714,0.5491753,0.460523752,0.75874789,F,0.1199025,0.1072939055,0.215667737,RV,CELLSTATES,vFB4,FB,control_TTN,6,14
7982043,ZYX,-1.9032982,5.071323,11.9903377,0.000810309,0.01913434,F,1.7181864,0.0418965648,0.022348622,RV,CELLSTATES,vFB4,FB,control_TTN,6,14
7982044,ZZEF1,0.3911716,6.774139,1.7548549,0.188516571,0.51901617,F,0.2848191,0.1629268748,0.319265485,RV,CELLSTATES,vFB4,FB,control_TTN,6,14


In [50]:
table(final_df[,'cell_state'])


                          AD2                      CD4T_act 
                        67076                        167690 
                   CD4T_naive                    CD8T_cytox 
                       100614                        100614 
                      CD8T_em                       CD8T_te 
                       134152                        100614 
                   CD8T_trans                          cDC2 
                       167690                        167690 
                        EC1.0                         EC2.0 
                       167690                        167690 
                        EC5.0                         EC6.0 
                       167690                        167690 
                        EC7.0                         EC8.0 
                       167690                        167690 
                         Mast                      MO_CD16. 
                       100614                        100614 
                     MO

In [51]:
tapply(final_df[,"FDR"], 
       final_df[,'cell_state'], 
       min)

AD2                      CD4T_act 
                 1.172938e-11                  2.318716e-19 
                   CD4T_naive                    CD8T_cytox 
                 2.126901e-04                  1.126201e-03 
                      CD8T_em                       CD8T_te 
                 1.741850e-03                  2.873929e-04 
                   CD8T_trans                          cDC2 
                 1.477687e-19                  1.642613e-12 
                        EC1.0                         EC2.0 
                 7.078201e-14                  1.288625e-12 
                        EC5.0                         EC6.0 
                 2.437715e-21                  5.437572e-15 
                        EC7.0                         EC8.0 
                 1.101870e-29                  2.635797e-45 
                         Mast                      MO_CD16. 
                 3.562862e-13                  5.304285e-07 
                     MO_VCAN.                      MP_FOLR2 
                 3.991006e-20                  4.863804e-08 
                       MP_ISG MP_LYVE1hi.MHCII_intermediate 
                 3.020082e-22                  5.023053e-19 
          MP_LYVE1hi.MHCII_lo           MP_LYVE1lo.MHCII_hi 
                 6.873844e-20                  1.237564e-20 
                       MP_OSM               MP_transitional 
                 7.968652e-34                  6.617015e-22 
                    MP_TREM2.                         NC1.0 
                 5.760998e-04                  1.583425e-06 
                        NC1.1                           NC2 
                 1.195040e-13                  4.386623e-09 
                    NK_CD16hi                     NK_CD56hi 
                 4.192044e-10                  4.908185e-08 
                          PC1                           PC2 
                 7.982795e-09                  1.116936e-40 
                          PC3                        SMC1.1 
                 7.455382e-17                  3.260953e-17 
                       SMC1.2                          SMC2 
                 8.129717e-25                  5.301672e-18 
                       vCM1.0                        vCM1.1 
                 2.842979e-07                  6.757448e-11 
                       vCM1.2                        vCM1.3 
                 4.456490e-18                  3.221759e-11 
                         vCM2                        vCM3.0 
                 1.054776e-05                  6.261554e-07 
                       vCM3.1                          vCM4 
                 9.662407e-08                  1.943686e-10 
                         vCM5                        vFB1.0 
                 5.063679e-15                  2.746608e-08 
                       vFB1.1                        vFB1.2 
                 2.422709e-33                  7.071880e-23 
                         vFB2                          vFB3 
                 9.455776e-11                  5.445674e-20 
                         vFB4 
                 4.166872e-16

In [52]:
write.csv(final_df, "/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALLCELLSTATES_ALLGENOTYPES_EDGER_RV_V8.csv")

In [53]:
head(final_df)

,Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison,Observations_genotype,Observations_reference
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,A1BG,1.2205588,4.281900,0.8834923,0.3472479,0.7936247,F,0.100384830,0.016013072,0.04910714,RV,CELLSTATES,AD2,AD,control_LMNA,4,6
2,A1BG-AS1,1.7543292,4.103563,1.6803753,0.1948759,0.6913935,F,0.160274735,0.005555556,0.02395833,RV,CELLSTATES,AD2,AD,control_LMNA,4,6
3,A1CF,-1.6756031,3.987134,1.1340045,0.3021312,0.7597694,F,0.119318197,0.027777778,0.00000000,RV,CELLSTATES,AD2,AD,control_LMNA,4,6
4,A2M,-0.1773584,6.034037,0.1038335,0.7472772,0.9772614,F,0.009989273,0.398457378,0.29449405,RV,CELLSTATES,AD2,AD,control_LMNA,4,6
5,A2M-AS1,-0.5136542,4.170491,0.1565951,0.6923112,0.9617922,F,0.016918769,0.027777778,0.01785714,RV,CELLSTATES,AD2,AD,control_LMNA,4,6
6,A2ML1,-2.0343873,4.098135,1.9439397,0.1632427,0.6913935,F,0.160274735,0.019394714,0.00000000,RV,CELLSTATES,AD2,AD,control_LMNA,4,6


---

# On cell-type level

## For RV

In [54]:
x=read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALL_CELLTYPE_PSEUDOBULK_RV.csv")
genes_tofilter=read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALL_CELLTYPE_PSEUDOBULK_FILTERING_RV.csv")
colnames(genes_tofilter) <- gsub("mutation.negative", 'PVneg', colnames(genes_tofilter))

# Removes patients with only few (<10) nuclei numbers (as they might bias the analysis)
CELLTYPE_FILTER <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ABSOLUTE_CELLTYPE_NUMBER_RV.csv")


rownames(x) <- x$X
x <- x[,-1]
l <- strsplit(colnames(x), "__")

In [55]:
# This steps are needed to make names equal - space and + was converted to .
#CELLTYPE_FILTER$cell_states <- gsub("\\+", "\\.", CELLTYPE_FILTER$cell_states)
#CELLTYPE_FILTER$cell_states <- gsub(" ", "\\.", CELLTYPE_FILTER$cell_states)
#CELLTYPE_FILTER$cell_states <- gsub("\\-", "\\.", CELLTYPE_FILTER$cell_states)

In [56]:
meta.data <- t(as.data.frame(l))
rownames(meta.data) <- colnames(x)

meta.data <- as.data.frame(meta.data)
colnames(meta.data)  <-  c("Genotype", "cell_type", 'Patient', 'X10X_version', 'Gender')

In [57]:
CELLTYPE_FILTER[,1][which(!(CELLTYPE_FILTER[,1] %in% meta.data$cell_type))]

factor(0)
Levels: AD CM EC FB Lymphoid Mast Mural Myeloid NC

In [58]:
# meta.data$cell_state <- gsub("CD4+T_act", "CD4.T_act", meta.data$cell_state)
# meta.data$cell_state <- gsub("CD4+T_naive", "CD4.T_naive", meta.data$cell_state)
# meta.data$cell_state <- gsub("CD4+T_reg", "CD4.T_reg", meta.data$cell_state)
# meta.data$cell_state <- gsub("CD8+T_cytox", "CD8.T_cytox", meta.data$cell_state)
# meta.data$cell_state <- gsub("CD8+T_em", "CD8.T_em", meta.data$cell_state)
# meta.data$cell_state <- gsub("CD8+T_te", "CD8.T_te", meta.data$cell_state)
# meta.data$cell_state <- gsub("CD8+T_trans", "CD8.T_trans", meta.data$cell_state)
# meta.data$cell_state <- gsub("CD16+ Mo", "CD16..Mo", meta.data$cell_state)
# meta.data$cell_state <- gsub("MAIT-like", "MAIT.like", meta.data$cell_state)
# meta.data$cell_state <- gsub("prolif Lymphoids", "prolif.Lymphoids", meta.data$cell_state)
# meta.data$cell_state <- gsub("prolif Myeloids", "prolif.Myeloids", meta.data$cell_state)

In [59]:
# Numbers of observations in total for Adipocytes, LMNA in RV
CELLTYPE_FILTER[1,as.character(unique(meta.data[which(meta.data$Genotype=="TTN"),"Patient"]))]

,H14,H36,H39,H41,H34,H35
,<int>,<int>,<int>,<int>,<int>,<int>
1,29,1,1,1,0,0


In [60]:
# Numbers of observations in total for Adipocytes, control in RV
CELLTYPE_FILTER[1,as.character(unique(meta.data[which(meta.data$Genotype=="control"),"Patient"]))]

,D1,D2,D4,D5,D6,D7,ED_H15,ED_H20,ED_H25,ED_H26,ED_H35,H49,H53,H55,ED_H37
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,10,2,38,3,12,13,1,80,10,89,25,97,19,4,0


In [61]:
head(meta.data)

,Genotype,cell_type,Patient,X10X_version,Gender
,<fct>,<fct>,<fct>,<fct>,<fct>
LMNA__AD__H06__V3__f,LMNA,AD,H06,V3,f
LMNA__AD__H24__V3__f,LMNA,AD,H24,V3,f
LMNA__AD__H25__V3__m,LMNA,AD,H25,V3,m
LMNA__AD__H27__V3__f,LMNA,AD,H27,V3,f
LMNA__AD__H28__V3__m,LMNA,AD,H28,V3,m
LMNA__AD__H29__V3__f,LMNA,AD,H29,V3,f


In [62]:
GENOTYPES <- unique(meta.data$Genotype[meta.data$Genotype != "control"])
GENOTYPES

[1] LMNA  PKP2  PVneg RBM20 TTN  
Levels: control LMNA PKP2 PVneg RBM20 TTN

In [63]:
CELL_LEVEL <- "CELLTYPE"
REGION <- "RV"

In [64]:
##########
# Convert to bool
CELLTYPE_FILTER <- CELLTYPE_FILTER
CELLTYPE_FILTER[,2:ncol(CELLTYPE_FILTER)] <- apply(CELLTYPE_FILTER[,-1], 2, function(i) i>=5)

In [65]:
unique(meta.data[which(meta.data$Genotype=="TTN"),"Patient"])

[1] H14 H36 H39 H41 H34 H35
42 Levels: D1 D2 D4 D5 D6 D7 ED_H15 ED_H20 ED_H25 ED_H26 ED_H35 ED_H37 ... IC_H04

In [66]:
# This is the example from above, they will be excluded in downstream analysis
# In this case, the comparison will be excluded, as only one sample has enough observations
CELLTYPE_FILTER[1,as.character(unique(meta.data[which(meta.data$Genotype=="TTN"),"Patient"]))]

,H14,H36,H39,H41,H34,H35
,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
1,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE


In [67]:
as.character(unique(meta.data$cell_type))

[1] "AD"       "CM"       "EC"       "FB"       "Lymphoid" "Mast"     "Mural"   
[8] "Myeloid"  "NC"

In [68]:
i <- 1

for(GENOTYPE in as.character(GENOTYPES)){
    message("\n###START: ", GENOTYPE," #####\n")
    
    for(CELL_STATE in as.character(unique(meta.data$cell_type))){
        
        CONTROL_COLUMN <- paste("control_",CELL_STATE, sep="")
        GENOTYPE_COLUMN <- paste(GENOTYPE,"_", CELL_STATE, sep="")
        CELLTYPE <- CELL_STATE #as.character(CELLTYPE_STATE[which(CELLTYPE_STATE$cell_type==CELL_STATE),"cell_type"])
        
        
        x_sub <- x[,which(meta.data$Genotype %in% c("control", GENOTYPE))]
        meta.data_sub <- meta.data[which(meta.data$Genotype %in% c("control", GENOTYPE)),]
        
        x_sub <- x_sub[,which(meta.data_sub$cell_type %in% c(CELL_STATE))]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$cell_type %in% c(CELL_STATE)),] 
        
        ##### remove first line (cell_type column)
        CELLTYPE_FILTER_SUB <- CELLTYPE_FILTER
        PATIENTS_TOKEEP <- CELLTYPE_FILTER_SUB[which(CELLTYPE_FILTER_SUB$cell_type==CELL_STATE),-1]
        PATIENTS_TOKEEP <- colnames(PATIENTS_TOKEEP)[which(as.logical(PATIENTS_TOKEEP))]
        #####
        
        # Subset only for patients which fulfill condition
        x_sub <- x_sub[,which(meta.data_sub$Patient %in% PATIENTS_TOKEEP)]
        meta.data_sub <- meta.data_sub[which(meta.data_sub$Patient %in% PATIENTS_TOKEEP),]
        meta.data_sub$Genotype = droplevels(meta.data_sub$Genotype) ## Drop factor levels
   
        
        # The test is only run, if 2 conditions have at least 2 patients
        if (length(unique(meta.data_sub$Genotype))==2 & 
            all(table(as.character(meta.data_sub$Genotype))>=3))
           {
             # Prepare DGEList object
            meta.data_sub$Genotype <- as.factor(as.character(meta.data_sub$Genotype))
            dge <- DGEList(counts=x_sub, group=meta.data_sub$Genotype)
            
            # Filter genes based on expression, the qlf Object will be filtered to exclude them from FDR calculation
            genes_tofilter_sub_GENOTYPE <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% GENOTYPE &
                                                                 gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE & 
                                                                 gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP)]

            genes_tofilter_sub_CONTROL <- genes_tofilter[,which(gsub("^(.*)__.*__.*$", "\\1", colnames(genes_tofilter)) %in% 'control' &
                                                                gsub("^.*__(.*)__.*$", "\\1", colnames(genes_tofilter)) %in% CELL_STATE & 
                                                                gsub("^.*__.*__(.*)$", "\\1", colnames(genes_tofilter)) %in% PATIENTS_TOKEEP)]
            
            genes_tofilter_sub <- data.frame("X"=genes_tofilter[,"X"],
                                             CONTROL_COLUMN=apply(genes_tofilter_sub_CONTROL, 1, mean),
                                             GENOTYPE_COLUMN=apply(genes_tofilter_sub_GENOTYPE, 1, mean))
            colnames(genes_tofilter_sub) <- c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)
            
            keep <- which(genes_tofilter_sub[,CONTROL_COLUMN] > 0.0125 |
                          genes_tofilter_sub[,GENOTYPE_COLUMN] > 0.0125)
            
            #keep <- genes_tofilter[,CONTROL_COLUMN]>0.0125 | ##### change & to | ##### 
            #        genes_tofilter[,GENOTYPE_COLUMN]>0.0125
            #dge <- dge[keep, , keep.lib.sizes=FALSE]
    
            # PP, model matrix, https://www.nature.com/articles/nmeth.4612 (edgeRQLFDetRate)
            dge <- calcNormFactors(dge)
            cdr <- scale(colMeans(x_sub > 0))
            design <- model.matrix(~ cdr + meta.data_sub$Genotype)
    
            # 
            dge <- estimateDisp(dge, design = design)
            fit <- glmQLFit(dge, design = design)
            qlf <- glmQLFTest(fit)
    
            # For all
            tt <- topTags(qlf, n = Inf)
            
            # Only for "expressed/detected" genes
            tt_filtered <- topTags(qlf[keep,], n = Inf)
            
            tt$table[,"Gene"] <- as.character(rownames(tt$table))
            tt_filtered$table[,"Gene"] <- as.character(rownames(tt_filtered$table))
            tt_filtered$table[,"low_expression"] <- "F" #####  Low-expression column ##### 
            tt_filtered$table[,"FDR_plot"] <- -log10(tt_filtered$table$FDR) #####  For plotting Volcano, here we use FDR, not pValue  ##### 
            
            
            tt_merged <- merge(tt$table[,c("Gene", "logFC", "logCPM", 'F', 'PValue')], 
                               tt_filtered$table[,c("Gene", "FDR", "low_expression", "FDR_plot")], 
                               by="Gene", all=T)   
            
            # Replace NA FDRs with NA                   #####  ORDER CHANGED #####  
            tt_merged[which(is.na(tt_merged$FDR)),"FDR"] <- 1 
            
            EXPRESSION_MEAN <- genes_tofilter_sub[,c("X", CONTROL_COLUMN, GENOTYPE_COLUMN)]
            colnames(EXPRESSION_MEAN) <- c("Gene", "mean_exp_control", 'mean_exp_genotype')
            tt_merged <- merge(tt_merged, EXPRESSION_MEAN, by="Gene")
    
            
            tt_merged[,"Region"] <- REGION
            tt_merged[,"annotation_level"] <- CELL_LEVEL
            tt_merged[,"cell_state"] <- CELL_STATE
            tt_merged[,"cell_type"] <- CELLTYPE
            tt_merged[,"comparison"] <- paste("control_", GENOTYPE, sep="")
            
            ###### Suggestion to add those columns
            tt_merged[,"Observations_genotype"] <- as.numeric(table(meta.data_sub$Genotype)[GENOTYPE])
            tt_merged[,"Observations_reference"] <- as.numeric(table(meta.data_sub$Genotype)['control'])
            
            # low expression add T
            tt_merged[which(is.na(tt_merged$low_expression)),"low_expression"] <- "T"
            
            if(i==1){
                 final_df <- tt_merged   
            } else {
                final_df <- rbind(final_df, tt_merged)
            }
            
            i <- i + 1
            message("\n###FINISHED: ", CELL_STATE," #####\n")
        } else {
            message("\n###Skip: ", CELL_STATE," #####\n")
        }
    }
}


###START: LMNA #####



###FINISHED: AD #####



###FINISHED: CM #####



###FINISHED: EC #####



###FINISHED: FB #####



###FINISHED: Lymphoid #####



###Skip: Mast #####



###FINISHED: Mural #####



###FINISHED: Myeloid #####



###FINISHED: NC #####



###START: PKP2 #####



###FINISHED: AD #####



###FINISHED: CM #####



###FINISHED: EC #####



###FINISHED: FB #####



###FINISHED: Lymphoid #####



###FINISHED: Mast #####



###FINISHED: Mural #####



###FINISHED: Myeloid #####



###FINISHED: NC #####



###START: PVneg #####



###FINISHED: AD #####



###FINISHED: CM #####



###FINISHED: EC #####



###FINISHED: FB #####



###FINISHED: Lymphoid #####



###Skip: Mast #####



###FINISHED: Mural #####



###FINISHED: Myeloid #####



###FINISHED: NC #####



###START: RBM20 #####



###FINISHED: AD #####



###FINISHED: CM #####



###FINISHED: EC #####



###FINISHED: FB #####



###FINISHED: Lymphoid #####



###FINISHED: Mast #####



###FINISHED: Mural #####





In [69]:
head(final_df)

,Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison,Observations_genotype,Observations_reference
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,A1BG,0.5928048,3.613484,0.4485169,0.5508307,0.8964962,F,0.04745154,0.011645189,0.017549262,RV,CELLTYPE,AD,AD,control_LMNA,4,10
2,A1BG-AS1,0.7467388,3.494549,0.8893044,0.4586736,1.0000000,T,NA,0.003185451,0.007628843,RV,CELLTYPE,AD,AD,control_LMNA,4,10
3,A1CF,-0.3983224,3.472606,0.1569457,0.7752702,1.0000000,T,NA,0.012154523,0.003164557,RV,CELLTYPE,AD,AD,control_LMNA,4,10
4,A2M,-0.4780581,5.389797,0.8279467,0.3683503,0.8103331,F,0.09133640,0.151950451,0.113212418,RV,CELLTYPE,AD,AD,control_LMNA,4,10
5,A2M-AS1,-0.7838415,3.559792,0.5894344,0.4471737,0.8525847,F,0.06926247,0.013619099,0.008620690,RV,CELLTYPE,AD,AD,control_LMNA,4,10
6,A2ML1,0.3085760,3.813569,0.1445228,0.7058489,0.9488013,F,0.02282472,0.010749557,0.017122514,RV,CELLTYPE,AD,AD,control_LMNA,4,10


In [70]:
tail(final_df)

,Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison,Observations_genotype,Observations_reference
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1408591,ZXDB,1.12076971,5.227782,4.54494520,0.1431496,1.0000000,T,NA,0.00000000,0.004504505,RV,CELLTYPE,NC,NC,control_TTN,6,15
1408592,ZXDC,0.41384666,7.213291,0.95890384,0.3274636,0.8265347,F,0.082738903,0.14185210,0.240512273,RV,CELLTYPE,NC,NC,control_TTN,6,15
1408593,ZYG11A,-1.13629790,5.304394,3.36949385,0.1663576,0.6800895,F,0.167433910,0.01607527,0.000000000,RV,CELLTYPE,NC,NC,control_TTN,6,15
1408594,ZYG11B,0.10468329,6.849860,0.05170007,0.8201311,0.9890962,F,0.004761453,0.11172083,0.129940881,RV,CELLTYPE,NC,NC,control_TTN,6,15
1408595,ZYX,-1.21363157,5.608672,2.25447012,0.1332296,0.6313218,F,0.199749238,0.02662245,0.007905865,RV,CELLTYPE,NC,NC,control_TTN,6,15
1408596,ZZEF1,0.09761451,7.200671,0.05165851,0.8202021,0.9890962,F,0.004761453,0.15385156,0.222772332,RV,CELLTYPE,NC,NC,control_TTN,6,15


In [71]:
write.csv(final_df, "/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALLCELLTYPES_ALLGENOTYPES_EDGER_RV_V8.csv")

In [72]:
x <- read.csv("/fast/AG_Huebner/huebner3/ANALYSES/20190926_gp_BadOyenhausen/scanpy/20210302/Global_touse/ALLCELLTYPES_ALLGENOTYPES_EDGER_RV_V8.csv")

In [87]:
tail(x)

,X,Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison,Observations_genotype,Observations_reference
,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>
1408591,1408591,ZXDB,1.12076971,5.227782,4.54494520,0.1431496,1.0000000,TRUE,NA,0.00000000,0.004504505,RV,CELLTYPE,NC,NC,control_TTN,6,15
1408592,1408592,ZXDC,0.41384666,7.213291,0.95890384,0.3274636,0.8265347,FALSE,0.082738903,0.14185210,0.240512273,RV,CELLTYPE,NC,NC,control_TTN,6,15
1408593,1408593,ZYG11A,-1.13629790,5.304394,3.36949385,0.1663576,0.6800895,FALSE,0.167433910,0.01607527,0.000000000,RV,CELLTYPE,NC,NC,control_TTN,6,15
1408594,1408594,ZYG11B,0.10468329,6.849860,0.05170007,0.8201311,0.9890962,FALSE,0.004761453,0.11172083,0.129940881,RV,CELLTYPE,NC,NC,control_TTN,6,15
1408595,1408595,ZYX,-1.21363157,5.608672,2.25447012,0.1332296,0.6313218,FALSE,0.199749238,0.02662245,0.007905865,RV,CELLTYPE,NC,NC,control_TTN,6,15
1408596,1408596,ZZEF1,0.09761451,7.200671,0.05165851,0.8202021,0.9890962,FALSE,0.004761453,0.15385156,0.222772332,RV,CELLTYPE,NC,NC,control_TTN,6,15


In [88]:
length(which(p.adjust(x[-which(is.na(x$FDR_plot)),"PValue"], method = 'BH')<0.05))

[1] 10977

In [89]:
x[-which(is.na(x$FDR_plot)),"FDR_plot_all"] <- p.adjust(x[-which(is.na(x$FDR_plot)),
                                                          "PValue"], 
                                                        method = 'BH')

In [90]:
x[which(x$FDR<1),]

,X,Gene,logFC,logCPM,F,PValue,FDR,low_expression,FDR_plot,mean_exp_control,mean_exp_genotype,Region,annotation_level,cell_state,cell_type,comparison,Observations_genotype,Observations_reference,FDR_plot_all
,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<dbl>
1,1,A1BG,0.59280479,3.613484,0.44851693,0.550830654,0.8964962,FALSE,0.047451535,0.011645189,0.01754926,RV,CELLTYPE,AD,AD,control_LMNA,4,10,0.8767031
4,4,A2M,-0.47805814,5.389797,0.82794671,0.368350327,0.8103331,FALSE,0.091336403,0.151950451,0.11321242,RV,CELLTYPE,AD,AD,control_LMNA,4,10,0.7785514
5,5,A2M-AS1,-0.78384150,3.559792,0.58943441,0.447173742,0.8525847,FALSE,0.069262471,0.013619099,0.00862069,RV,CELLTYPE,AD,AD,control_LMNA,4,10,0.8266161
6,6,A2ML1,0.30857597,3.813569,0.14452276,0.705848923,0.9488013,FALSE,0.022824715,0.010749557,0.01712251,RV,CELLTYPE,AD,AD,control_LMNA,4,10,0.9370670
7,7,A2ML1-AS1,0.57931432,5.658915,1.74265342,0.194352982,0.6787966,FALSE,0.168260364,0.156997997,0.24860482,RV,CELLTYPE,AD,AD,control_LMNA,4,10,0.6329451
9,9,A3GALT2,-2.43383781,3.526761,5.50740195,0.049297917,0.4312155,FALSE,0.365305621,0.016359989,0.00000000,RV,CELLTYPE,AD,AD,control_LMNA,4,10,0.3699974
10,10,A4GALT,-1.52364885,4.982904,6.05813642,0.018280381,0.3050396,FALSE,0.515643730,0.140615087,0.03783633,RV,CELLTYPE,AD,AD,control_LMNA,4,10,0.2375570
12,12,AAAS,-0.46603918,4.709302,0.58753257,0.447902270,0.8529603,FALSE,0.069071167,0.084879806,0.05538560,RV,CELLTYPE,AD,AD,control_LMNA,4,10,0.8270879
13,13,AACS,-0.98908185,7.550449,5.26392754,0.027132919,0.3449701,FALSE,0.462218499,1.043062838,0.52544506,RV,CELLTYPE,AD,AD,control_LMNA,4,10,0.2840057
